In [ ]:
import os
import tarfile
from six.moves import urllib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Load and Read MNIST

In [ ]:
URL_MNIST = r"https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat"
PATH_MNIST = r"E:\jupyter_code\datasets\mldata"
# def fetch(url_mnist=URL_MNIST, path_mnist=PATH_MNIST):
#     if not os.path.isdir(PATH_MINST):
#         os.makedirs(PATH_MINST)
#     FILE_PATH = os.path.join(PATH_MINST, "mnist-original.mat")
#     urllib.request.urlretrieve(URL_MNIST, FILE_PATH)

# from sklearn.datasets.base import get_data_home
# print(get_data_home())
from sklearn.datasets import fetch_mldata
from sklearn.datasets.base import get_data_home, load_data
FILE_PATH = r"E:\jupyter_code\datasets"
digits = fetch_mldata("MNIST original", data_home=FILE_PATH)
# print(digits.data.shape)
# temp = digits.data[0].reshape(28, 28)
# plt.matshow(temp, cmap=plt.cm.gray_r)
# plt.show()

In [ ]:
X, y = digits.data, digits.target
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
permute = np.random.permutation(60000)
X_train, y_train = X_train[permute], y_train[permute]

# 二分类

In [ ]:
# 划分数据
y_train_5, y_test_5 = (y_train == 5), (y_test == 5)
# SGD 分类
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3)
sgd_clf.fit(X_train, y_train_5)
# 交叉验证
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=5, n_jobs=-1)

In [ ]:
# 评价模型
# T:True, F:False, P:Positive, N:Negative
# recall = TP / (TP + FN)  在逃犯信息检索系统中，更希望尽可能地少漏掉逃犯，此时，就要提高召回率（查全率）
# precision = TP / (TP + FP)  商品推荐系统中，为了尽可能地少打扰用户，更希望推荐内容确实是用户感兴趣的，就要提高精度（查准率）
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_train_pred, y_train_5)
recall = recall_score(y_train_pred, y_train_5)
f1 = f1_score(y_train_pred, y_train_5)

In [ ]:
# 不同 thresholds 下，precision, recall 的变化曲线
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=5, n_jobs=-1, method="decision_function")
precision, recall, thresholds = precision_recall_curve(y_train_5, y_scores)

plt.plot(thresholds, precision[:-1], "b--", label="Precision")
plt.plot(thresholds, recall[:-1], "g-", label="Recall")
plt.xlabel("Thresholds")
plt.legend(loc="best")
plt.ylim([0,1])
plt.show()

In [ ]:
# ROC 曲线，AUC 面积
fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)
auc(fpr, tpr)
roc_auc_score(y_train_5, y_scores)

# 多分类
#### OVR： 把某个类别的样本归为一类,其他剩余的样本归为另一类，构造出了k个SVM。将未知样本分类为具有最大分类函数值的那类。
#### 优点： 训练k个分类器，个数较少，其分类速度相对较快。
#### 缺点： 每个分类器的训练都是将全部的样本作为训练样本，这样在求解二次规划问题时，训练速度会随着训练样本的数量的增加而急剧减慢；
#### 负类样本要远远大于正类样本，出现样本不对称，且随着训练数据的增加而趋向严重。可以引入不同的惩罚因子，较少的正类采用较大的惩罚因子C；
#### 还有就是当有新的类别加进来时，需要对所有的模型进行重新训练。
#### 衍生： 决策树
#### OVO： 在任意两类样本之间设计一个SVM，k个类别的样本需要k(k-1)/2个SVM。对未知样本分类时得票最多的类别为该未知样本的类别。
#### 注： 仅使用这两类样本。
#### 优点： 不需要重新训练所有的SVM，只需要重新训练和增加语音样本相关的分类器。在训练单个模型时，相对速度较快。
#### 缺点： 所需构造和测试的二值分类器的数量关于k成二次函数增长，总训练时间和测试时间相对较慢。
#### 衍生： 有向无环图

In [ ]:
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(max_iter=100, tol=1e-3)  # hinge: SVM
sgd_clf.fit(X_train, y_train)  # OVO: SVM 对大数据支持不好
# 预测
predict = sgd_clf.predict([X[62000]])
score = sgd_clf.decision_function([X[62000]])

In [ ]:
# 交叉验证
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
X_train_scale = StandardScaler().fit_transform(X_train.astype(np.float64))
cross_val_score(sgd_clf, X_train_scale, y_train, scoring="accuracy", cv=4, n_jobs=-1)

# 错误分析： 混淆矩阵法

In [ ]:
# 第 i 行第 j 列的数字代表数字 i 被预测为数字 j 的个数总和
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(sgd_clf, X_train_scale, y_train, cv=5, n_jobs=-1)
confusion = confusion_matrix(y_train, y_pred)
confusion

In [ ]:
# 显示以及标准化
import matplotlib.pyplot as plt
plt.matshow(confusion, cmap=plt.cm.gray)  # 显示正确的分类数
# 每一类除以相应类别总数。比较错误率，而不是绝对的错误数，这对大的类别不公平
row_sum = confusion.sum(axis=1, keepdims=True)
norm_conf = confusion / row_sum.astype(np.float64)  
np.fill_diagonal(norm_conf, 0)
plt.matshow(norm_conf, cmap=plt.cm.gray)
plt.show()

# 多标签分类

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# 多标签k邻近, ML-kNN, 得到距离它最近的k个实例，得到这些实例的标签集合，通过最大后验概率准则来确定新实例的标签集合
y_multi_label = np.c_[y_train >= 7, np.bitwise_and(y_train.astype(np.int8), 1)]
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train_scale, y_multi_label)
knn_clf.predict([X[62000]]), y[62000]

In [ ]:
# 多输出分类 （去噪）
from matplotlib.pyplot import matshow, imshow
# 加噪
noise = np.random.randint(0, 100, size=X_train.shape)
X_noise_train = X_train + noise
y_noise_train = X_train
# 去噪
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_noise_train, y_noise_train)
# 做图
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 8))
idx = 36000
combine = zip([X_noise_train[idx], y_noise_train[idx], knn_clf.predict([X_noise_train[idx]])], ["train", "label", "predict"])
for i, (data, label) in enumerate(combine):
    ax[i].imshow(data.reshape(28,28), cmap=plt.cm.gray)
    ax[i].set_xlabel(label)
plt.show()